***

## Multitape Turing Machines

>csc427: Theory of Automata and Complexity. 
<br>
university of miami
<br>
spring 2020.
<br>
Burton Rosenberg.
<br>
<br>
Created: 24 March 2020
<br>last update: 24 March 2020

***


### The Plan

There are variants of Turing Machines that make Turing Machine Programming simpler. However we wish to proove that these variants just add convenince for the programmer, but not power. This is done by simulating the variant TM with a standard TM. Since whatever the variant can do a standard TM can do, by the simulation, their powers are the same.

The two variants of interest are mutliple tapes and nondeterminism. 

A multitape TM has more than one tape, with an independent head on each tape; and the transitions take into consideration the symbols all tapes, and can write and act on each tape. For instance, a one tape TM as transitions:
 
 (state, symbol) &rarr; ( state, action, symbol )
 
 a 2-tape TM would have transitions:
 
  (state, symbol_1, symbol_2) &rarr; ( state, action_1, symbol_1, action_2, symbol_2 )
  
**The tape format: from many, one**

- Turn k-tapes into one by interleaving the contents onto the single tape.
- Simulate the k-heads by having k markers, say 1, 2, ..., k, that preceeds each of the
  heads on the interleaved tapes. 
- To do the above, each cell on the any of the k-tapes go to two cells on the interleaved.
  The first cell reserved for the head marker (or blank if the head is not there) and the 
  second for the symbol.
- The structure above is called the tape area on the interleaved tape.
- Before the tape area are the state area and the staging area. 
- The staging are is in-out; for in, fetch the k symbols under the k heads and copy them
  to the staging area; for out, the staging area will contain the k pairs of action and 
  symbols. These were distributed into the tape area.
- The state area gives the state of the simulated machine. For uniformity, states will be
  numbered, and the state area is fixed size large enough to hold the largest
  state number, written in binary.
  
**Transitions: ch-ch-ch-changes**

The original k-tape machine as transitions such as (for 2-tapes):

 (state, symbol_1, symbol_2) &rarr; ( state, action_1, symbol_1, action_2, symbol_2 )
 
Once we have copied the current symbols into the staging area, the head of the 
combined tape begins with:

  :&lt;state&gt;: x y:

where the angle brackets means a left justified binary representation of the current
state number, and x and y being the tape1 and tape2 symbols currently under their 
respectived heads.

The transition is extracted from the finite state of the simulating machine by
it's walking this tape prefix brancing with each step. When this is achieved, the 
machine is an a state which is essentially a leaf of a tree. This leaf is exactly
identifies the transition to apply. Appened to that leave then is a set of fixed
state transitions at write of the state and staging area with the output of the
transition.

In this way, the original machine is encoded into a big (but finite) tree. The paths
of the tree decode state and tape symbols; and that decode is followed immediately 
with a write of the output of the state.

**Putting it together**

So simulation repeats the three phases:

1. Fetch the current symbols into the staging area.
2. Run the state transition tree to write into the state area and staging area.
3. File away the actions and symbols written into the staging area into the tape area.




In [1]:
import string
import sys
import os
import argparse
import re

#
# tm-sim.py
#
# author: bjr
# date: 21 mar 2020
# last update: 22 mar 2020
#
# copyright: Creative Commons. See http://www.cs.miami.edu/home/burt
#


#
# BETA VERSION .. RELOAD OFTEN 
#

class TuringMachine:

    def __init__(self,verbose="none",endmarker=False):
        self.start_state = ""
        self.accept_states = set()
        self.reject_states = set()
        self.transitions = {}
        self.current_state = ""
        self.step_counter = 0
        self.all_actions = ["r","l","n"]
        self.verbose_levels = {"none":0, "verbose":1, "debug":2}
        self.tape = [' ']
        self.position = 0
        self.verbose = self.verbose_levels[verbose]
        self.endmarker = endmarker

    def set_start_state(self,state):
        self.start_state = state

    def set_tape(self,tape_string):
        # change '_' to ' '
        self.tape = [' ' if symbol=='_' else symbol 
                         for symbol in tape_string]
        if self.endmarker:
            self.tape.insert(0,':')
        
    def set_verbose(self,verbose):
        self.verbose = 0
        if verbose in self.verbose_levels:
            self.verbose = self.verbose_levels[verbose]

    def set_endmarker(self,endmarker):
        self.endmarker = endmarker

    def add_accept_state(self,state):
        self.accept_states.add(state)

    def add_reject_state(self,state):
        self.reject_states.add(state)
    
    def get_current_state(self):
        return self.curent_state

    def add_transition(self,state_from,read_symbol,
                       write_symbol,action,state_to):
        """
        Returns None on success; else return an error string.
        """
        
        if self.verbose >= self.verbose_levels['debug']:
            print("adding transition:", 
                  state_from, read_symbol, write_symbol, action, state_to )

        if read_symbol =='_': 
            read_symbol = ' '
        if write_symbol =='_':
            write_symbol = ' '

        if action.lower() not in self.all_actions:
            # return something instead, nobody likes a chatty program
            return "WARNING: unrecognized action, skipping."
        x = (state_from, read_symbol)
        if x in self.transitions:
            return "WARNING: multiple outgoing states not allowed for DFA's, skipping."
        self.transitions[x] = (state_to,write_symbol,action)
        return None

    def restart(self,tape_string):
        self.current_state = self.start_state
        self.position = 0
        if len(tape_string)==0 :
            tape_string = ' '
        self.set_tape(tape_string)
        self.step_counter = 1

    def step_transition(self):
        """
        take one state transition, based on tape, states, and transitions.
        Returns None if ok; else returns unmatched transition
        """
        c_s = self.current_state
        x = (c_s,self.tape[self.position])
        if x in self.transitions:
            (new_state, symbol, action ) = self.transitions[x]
        else:
            if self.verbose>=self.verbose_levels['debug']:
                print('current state:', c_s, 'current symbol: |', 
                      self.tape[self.position],'| current position: ', self.position)
            return str(x)
        self.current_state = new_state
        self.tape[self.position] = symbol

        shout = False
        if action.lower() != action:
            shout = True
            action = action.lower()
        
        if action == 'l' and self.position>0:
            self.position -= 1
        if action == 'r':
            self.position += 1
            if self.position==len(self.tape):
                self.tape[self.position:] = ' '
        if action == 'n':
            pass
   
        if shout:
            self.print_tape()

        if self.verbose >= self.verbose_levels['debug']:
            print("\t", self.step_counter, "\t", new_state, symbol, action)
        self.step_counter += 1
        return None

    def compute_tm(self,tape_string,step_limit=0):
        self.restart(tape_string)
        step = 0
            
        stop_states = self.accept_states.union(self.reject_states)
        while self.current_state not in stop_states:
            res = self.step_transition()
            if res:
                return ("no transition",res)
            step += 1
            if step > step_limit:
                return ("step limit",step,''.join(self.tape))
            if self.verbose >= self.verbose_levels['debug']:
                print(step, self.current_state, self.position, self.tape )

        cause = "reject"
        if self.current_state in self.accept_states:
            cause = "accept"
        the_tape = ''.join(self.tape)
        return (cause,the_tape)

    def print_tape(self):
        t, p = self.tape, self.position
        s = ''.join(t[:p] + ['_'] + [t[p]] + ['_'] + t[p+1:])
        print("step:",self.step_counter, "state:", self.current_state,"\t",s)
    
    def print_tm(self):
        print("\nstart state:\n\t",self.start_state)
        print("accept states:\n\t",self.accept_states)
        print("reject states:\n\t",self.reject_states)
        print("transitions:")
        for t in self.transitions:
            print("\t",t,"->",self.transitions[t])
        # print("tape:\n\t",self.tape)
        
### end class TuringMachine


class MachineParser:

    @staticmethod
    def turing(tm_obj, fa_string):
        """
        Code to parse a Turing Machine description into the Turing Machine object.
        """
        
        fa_array = fa_string.splitlines()
        line_no = 0 
        current_state = ""
        in_state_read = False
        in_accept_read = False
        in_reject_read = False

        for line in fa_array:
            while True:

                # comment lines are fully ignored
                if re.search('^\s*#',line):
                    break

                if re.search('^\s+',line):

                    if in_state_read:
                        m = re.search('\s+(\w|:)\s+(\w|:)\s+(\w)\s+(\w+)',line)
                        if m:
                            res = tm_obj.add_transition(current_state,
                                    m.group(1),m.group(2),m.group(3),m.group(4))
                            if res: 
                                print(res)
                            break

                    if in_accept_read:
                        m = re.search('\s+(\w+)',line)
                        if m:
                            tm_obj.add_accept_state(m.group(1))
                            break

                    if in_reject_read:
                        m = re.search('\s+(\w+)',line)
                        if m:
                            tm_obj.add_reject_state(m.group(1))
                            break

                in_state_read = False
                in_accept_read = False
                in_reject_read = False

                # blank lines do end multiline input
                if re.search('^\s*$',line):
                    break ;

                m = re.search('^start:\s*(\w+)',line)
                if m:
                    tm_obj.set_start_state(m.group(1))
                    break

                m = re.search('^accept:\s*(\w+)',line)
                if m:
                    tm_obj.add_accept_state(m.group(1))
                    in_accept_read = True
                    break

                m = re.search('^reject:\s*(\w+)',line)
                if m:
                    tm_obj.add_reject_state(m.group(1))
                    in_reject_read = True
                    break

                m = re.search('^state:\s*(\w+)',line)
                if m:
                    in_state_read = True
                    current_state = m.group(1)
                    break

                print(line_no,"warning: unparsable line, dropping: ", line)
                break

            line_no += 1
        return

### end class MachineParser


def create_and_test_turing_machine(tm_description, test_cases, 
                                   verbose='none', endmarker=False):
    tm = TuringMachine(verbose,endmarker)
    MachineParser.turing(tm,tm_description)
    
    print("\n\n*** THE TURING MACHINE ***")
    tm.print_tm()

    print("\n\n*** TEST RUNS ***\n\n")

    for s in test_cases:
        print("input:",s)
        # assume complexity is some quadratic
        print(tm.compute_tm(s,step_limit=10*(len(s)+5)**2))
    print("\n\n*** RUN COMPLETE ***\n\n")

def create_and_iterate_turing_machine(tm_description, starting_tape, count, 
                                   verbose='none', endmarker=False):
    
    tm = TuringMachine(verbose,endmarker)
    MachineParser.turing(tm,tm_description)
    
    print("\n\n*** THE TURING MACHINE ***")
    tm.print_tm()

    print("\n\n*** COUNT RUNS ***\n\n")

    tape = starting_tape
    for i in range(count):
        # assume complexity is some quadratic
        tape = tm.compute_tm(tape,step_limit=10*(len(tape)+5)**2)
        tape = tape[1][1:]

    print("\n\n*** RUN COMPLETE ***\n\n")
    

### Multitape

In [26]:
mt1="""# Simulating k-tapes a on 1-tape TM
# for this example, reduce to 2 tapes, with tape alphabet a and b.
#
# the full tape has three sections.
# (1) a fixed length string recording the state of the machine under simulation
# (2) a staging area into which are copied the current tape symbols, to simulate
#     the transition, and into which are copied the the output of the transition, 
#     before the symbol and action are recorded in the tape area
# (3) the tape area, in which the two tapes are interleaved, and a place is left
#     to mark "here!" for each of the heads.
#
#   :{0|1|_}^n:{r|l|n|_}{a|b|_}{r|l|n|_}{a|b|_}({_|1}{a|b}{_|2}{a|b})+
#
# there must be exactly one 1 and exactly one 2 on the tape (the head must be somewhere)
#

#
# we start simpler though, 
#    :={a|b|_}{a|b|_}({_|1}{a|b}{_|2}{a|b})+
#

start: s
accept: a
reject: r

state: s
    : : n find_head_t1

#
# bringing the state symbols to the beginning of the tape
#

state: find_head_t1
    : : r find_head_t1
    _ _ r find_head_t1
    a a r find_head_t1
    b b r find_head_t1
    2 2 r find_head_t1
    1 1 r head_t1_found
 
state: head_t1_found
    a a l rtrn_a_t1
    b b l rtrn_b_t1
    _ _ l rtrn___t1

state: rtrn_a_t1
    _ _ l rtrn_a_t1
    a a l rtrn_a_t1
    b b l rtrn_a_t1
    1 1 l rtrn_a_t1
    2 2 l rtrn_a_t1
    : : r wrt_a_t1
state: rtrn_b_t1
    _ _ l rtrn_b_t1
    a a l rtrn_b_t1
    b b l rtrn_b_t1
    1 1 l rtrn_b_t1
    2 2 l rtrn_b_t1
    : : r wrt_b_t1
state: rtrn___t1
    _ _ l rtrn___t1
    a a l rtrn___t1
    b b l rtrn___t1
    1 1 l rtrn___t1
    2 2 l rtrn___t1
    : : r wrt___t1

state: wrt_a_t1
    a a r find_head_t2
    b a r find_head_t2
    _ a r find_head_t2
state: wrt_b_t1
    a b r find_head_t2
    b b r find_head_t2
    _ b r find_head_t2
state: wrt___t1
    a _ r find_head_t2
    b _ r find_head_t2
    _ _ r find_head_t2

state: find_head_t2
    : : r find_head_t2
    _ _ r find_head_t2
    a a r find_head_t2
    b b r find_head_t2
    1 1 r find_head_t2
    2 2 r head_t2_found

state: head_t2_found
    a a l rtrn_a_t2
    b b l rtrn_b_t2
    _ _ l rtrn___t2

state: rtrn_a_t2
    _ _ l rtrn_a_t2
    a a l rtrn_a_t2
    b b l rtrn_a_t2
    1 1 l rtrn_a_t2
    2 2 l rtrn_a_t2
    : : r wrt_a_t2
state: rtrn_b_t2
    _ _ l rtrn_b_t2
    a a l rtrn_b_t2
    b b l rtrn_b_t2
    1 1 l rtrn_b_t2
    2 2 l rtrn_b_t2
    : : r wrt_b_t2
state: rtrn___t2
    _ _ l rtrn___t2
    a a l rtrn___t2
    b b l rtrn___t2
    1 1 l rtrn___t2
    2 2 l rtrn___t2
    : : r wrt___t2

state: wrt_a_t2
    a a r wrt_a_t2_x
    b b r wrt_a_t2_x
    _ _ r wrt_a_t2_x
state: wrt_a_t2_x
    a a r rtrn_stop
    b a r rtrn_stop
    _ a r rtrn_stop
state: wrt_b_t2
    a a r wrt_b_t2_x
    b b r wrt_b_t2_x
    _ _ r wrt_b_t2_x
state: wrt_b_t2_x
    a b r rtrn_stop
    b b r rtrn_stop
    _ b r rtrn_stop
state: wrt___t2
    a a r wrt___t2_x
    b b r wrt___t2_x
    _ _ r wrt___t2_x
state: wrt___t2_x
    a _ r rtrn_stop
    b _ r rtrn_stop
    _ _ r rtrn_stop

# mission accomplished

state: rtrn_stop
    a a l rtrn_stop
    b b l rtrn_stop
    _ _ l rtrn_stop
    1 1 l rtrn_stop
    2 2 l rtrn_stop
    : : n a

"""

def combine_two_tapes(tape1,h1,tape2,h2):
    if (h1>=len(tape1)) :
        h1 = 0
    if (h2>=len(tape2)) :
        h2 = 0
    tape1 += '  '
    tape2 += '  '
    tape1 += ' '*len(tape2)
    tape2 += ' '*len(tape1)
    tape = ":  "
    
    tape_pos = 0
    for t1s, t2s in zip(tape1,tape2):
        head1 = ' '
        if tape_pos == h1:
            head1 = '1'
        head2 = ' '
        if tape_pos == h2:
            head2 = '2'
        tape += (head1+t1s+head2+t2s)
        tape_pos += 1

    return tape

tape1 = "aaa"
tape2 = "bbbbb"
tt = []
for i in range(len(tape1)):
    for j in range(len(tape2)):
        tt += [combine_two_tapes("aaa",i,"bbbbb",j)]
print(tt)
create_and_test_turing_machine(mt1,tt)

[':  1a2b a b a b   b   b                            ', ':  1a b a2b a b   b   b                            ', ':  1a b a b a2b   b   b                            ', ':  1a b a b a b  2b   b                            ', ':  1a b a b a b   b  2b                            ', ':   a2b1a b a b   b   b                            ', ':   a b1a2b a b   b   b                            ', ':   a b1a b a2b   b   b                            ', ':   a b1a b a b  2b   b                            ', ':   a b1a b a b   b  2b                            ', ':   a2b a b1a b   b   b                            ', ':   a b a2b1a b   b   b                            ', ':   a b a b1a2b   b   b                            ', ':   a b a b1a b  2b   b                            ', ':   a b a b1a b   b  2b                            ']


*** THE TURING MACHINE ***

start state:
	 s
accept states:
	 {'a'}
reject states:
	 {'r'}
transitions:
	 ('s', ':') -> ('find_head_t1', ':', 'n')
	 ('find_head_t1', ':') -